In [1]:
# importing libraries
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

from PyQt5 import QtWidgets, uic , QtGui
from PyQt5.QtCore import *
from PyQt5.QtGui import *
from PyQt5.QtWidgets import *

import sys , warnings , requests
warnings.filterwarnings("ignore")

In [2]:
class Instagram:
    def __init__(self):
        chrome_options = webdriver.ChromeOptions() 
        prefs = {"profile.default_content_setting_values.notifications":2,"profile.managed_default_content_settings.images": 2} #parametrs to disable asking notification and loading images
        chrome_options.add_experimental_option("prefs", prefs) # set prefs
        self.driver = webdriver.Chrome(options =chrome_options) #initial chrome driver
        self.driver.set_page_load_timeout(10) #set page load time to 10 seconds    
        
        login_state = False
        while login_state == False:
            login_state = self.login()
            print(login_state)
        
        self.Not_now_button() 

        self.profile = False
        while self.profile == False:
            self.profile = self.get_info()
            

    def login(self):
        """ login to instagram main page by username and password"""
        try:
            self.driver.get("https://www.instagram.com/?hl=en")
            #driver.find_element(by=By.NAME, value='username').click()

            input_username_Element = self.driver.find_element(by=By.NAME, value='username') #find username input
            input_username_Element.send_keys('insta_for_sel_ap@hi2.in') #type username

            input_password_Element = self.driver.find_element(by=By.NAME, value='password') #find username password
            input_password_Element.send_keys('AP_1401@sel') #type password

            #driver.find_element(by=By.CSS_SELECTOR, value='.sqdOP > .qF0y9').click()   
            WebDriverWait(self.driver, 10).until(EC.visibility_of_element_located((By.XPATH, "/html/body/div[1]/div/div[1]/div/div[1]/div/div/div[1]/div[1]/section/main/div/header/section/div[0]"))).click() #click on login button
            return True
        except:
            return False
        
    def Not_now_button(self):
        try:
            WebDriverWait(self.driver, 10).until(EC.visibility_of_element_located((By.XPATH, "/html/body/div[1]/div/div[1]/div/div[1]/div/div/div[1]/div[1]/section/main/div/header/section/div[1]"))).click() #click on Not now button on Save Your Login Info
            return True
        except: 
            return False
        
    def get_info(self):
        try:
            self.driver.get("https://www.instagram.com/insta_for_sel_ap/")
            post_count = WebDriverWait(self.driver, 5).until(EC.visibility_of_element_located((By.XPATH, "//div[text()=' posts']"))).text
            try: name = self.driver.find_element(By.XPATH, "/html/body/div[1]/div/div[1]/div/div[1]/div/div/div[1]/div[1]/section/main/div/header/section/div[2]/span").text
            except: name = None
            try: bio = self.driver.find_element(By.XPATH, "/html/body/div[1]/div/div[1]/div/div[1]/div/div/div[1]/div[1]/section/main/div/header/section/div[2]/div").text
            except: bio = None    
            try: id_ = self.driver.find_element(By.XPATH, "/html/body/div[1]/div/div[1]/div/div[1]/div/div/div[1]/div[1]/section/main/div/header/section/div[2]/div").text
            except: id_ = None        
            return   {
            'post_count': post_count,#driver.find_element(By.XPATH, "/html/body/div[1]/div/div[1]/div/div[1]/div/div/div[1]/div[1]/section/main/div/header/section/ul/li[1]/div/span").text,
            'followers_count': self.driver.find_element(By.XPATH, "//div[text()=' followers']").text, 
            'following_count': self.driver.find_element(By.XPATH, "//div[text()=' following']").text,
            'name': name,
            'bio': bio,
            'id': id_,
            }
        except:
            return False
        
    def send_direct(self,id_contanct ,msg):
        self.driver.get(f"https://www.instagram.com/{id_contanct}/")

        WebDriverWait(self.driver, 10).until(EC.visibility_of_element_located((By.XPATH, "//div[text()='Message']"))).click()
        try:
            input_message_Element = WebDriverWait(self.driver, 10).until(EC.visibility_of_element_located((By.XPATH, "//textarea[@placeholder='Message...']")))
            input_message_Element.click()
            input_message_Element.send_keys(msg)
            input_message_Element.send_keys(Keys.ENTER)
            return True
        except Exception as e: 
            return False
        
    def get_scr(self):
        try:
            return {"type":"video","url": self.driver.find_element(By.XPATH, "//video[starts-with (@poster,'data')]").find_element(By.XPATH,'//source').get_attribute("src")}
        except:
            try:
                return {"type":"img","url": self.driver.find_element(By.XPATH, "/html/body/div[1]/div/div[1]/div/div[1]/div/div/div[1]/div[1]/section/div[1]/div/div[6]/div/div/img").get_attribute("src")}
            except: 
                return {"type":"unknow","url":"Cant find url"}
            
    def get_stories(self):
        list_data = []
        try:
            self.driver.get("https://www.instagram.com/")

            while self.driver.current_url == 'https://www.instagram.com/':
                try:
                    WebDriverWait(self.driver, 10).until(EC.visibility_of_element_located((By.XPATH,"//button[starts-with (@aria-label,'Story by')]"))).click()
                except:
                    pass

            while self.driver.current_url != 'https://www.instagram.com/':
                try:
                    list_data.append(self.get_scr())
                    WebDriverWait(self.driver, 5).until(EC.visibility_of_element_located((By.XPATH, "//button[@aria-label='Next']"))).click()
                except:
                    self.driver.find_elements(By.CSS_SELECTOR, ".coreSpriteRightChevron")[0].click()

            return list_data
        except:
            return False

In [3]:

# function to alter image
def mask_image(imgdata, imgtype ='png', size = 175):

	# Load image
	image = QImage.fromData(imgdata, imgtype)

	# convert image to 32-bit ARGB (adds an alpha
	# channel ie transparency factor):
	image.convertToFormat(QImage.Format_ARGB32)

	# Crop image to a square:
	imgsize = min(image.width(), image.height())
	rect = QRect(
		(image.width() - imgsize) / 2,
		(image.height() - imgsize) / 2,
		imgsize,
		imgsize,
	)
	
	image = image.copy(rect)

	# Create the output image with the same dimensions
	# and an alpha channel and make it completely transparent:
	out_img = QImage(imgsize, imgsize, QImage.Format_ARGB32)
	out_img.fill(Qt.transparent)

	# Create a texture brush and paint a circle
	# with the original image onto the output image:
	brush = QBrush(image)

	# Paint the output image
	painter = QPainter(out_img)
	painter.setBrush(brush)

	# Don't draw an outline
	painter.setPen(Qt.NoPen)

	# drawing circle
	painter.drawEllipse(0, 0, imgsize, imgsize)

	# closing painter event
	painter.end()

	# Convert the image to a pixmap and rescale it.
	pr = QWindow().devicePixelRatio()
	pm = QPixmap.fromImage(out_img)
	pm.setDevicePixelRatio(pr)
	size *= pr
	pm = pm.scaled(size, size, Qt.KeepAspectRatio,Qt.SmoothTransformation)

	# return back the pixmap data
	return pm

In [5]:
ali = Instagram()

True


In [ ]:
class Ui(QtWidgets.QMainWindow):
    def __init__(self):
        super(Ui, self).__init__()
        uic.loadUi('untitled1.ui', self)

        self.setWindowTitle("Instagram")
        
        # image path
        imgpath = "image.jpg"
        self.info = ali.get_info()
        # loading image
        r = requests.get(self.info['profile_photo'])
        if r.status_code == 200:
            with open(imgpath, 'wb') as f:
                for chunk in r:
                    f.write(chunk)
        imgdata = open(imgpath, 'rb').read()
        pixmap = mask_image(imgdata)
        self.ilabel = self.findChild(QtWidgets.QLabel, 'Image')
        self.ilabel.setPixmap(pixmap)
        self.findChild(QtWidgets.QPushButton, 'ask_path').clicked.connect(self.saveFileDialog)

        self.show()

    def saveFileDialog(self):
        #options = QFileDialog.Options()
        #options |= QFileDialog.DontUseNativeDialog
        #global folder_selected
        dialog = QFileDialog()
        foo_dir = dialog.getExistingDirectory(self, 'Select an awesome directory')
        print(foo_dir)
        
if __name__ == '__main__':
    app = QtWidgets.QApplication(sys.argv)
    w = Ui()
    sys.exit(app.exec_())

In [ ]:
ali.profile